# Model Evaluation
This is a modified version of the Notebook [object_detection.ipynb](https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb) from TensorFlow.

# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import warnings
import timeit

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# Object detection imports
warnings.filterwarnings('ignore') # "Real" coders ignore all warnings. Joke apart: matpotlib shows huge warning here.
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
warnings.filterwarnings('always') # Switch warning on again (you'll never know...)

# This is needed to display the images.
%matplotlib inline

In [ ]:
# Check correct TF version (1.5+ is needed for the models)
if tf.__version__ < '1.5.0':
    print('You are using TF version', tf.__version__)
    raise ImportError('Please upgrade your tensorflow installation to v1.5.* or later!')

# Model preparation 

## Variables

In [ ]:
# What models to download & test.
MODELS = ['ssd_mobilenet_v1_coco_2017_11_17', 
          'faster_rcnn_inception_v2_coco_2018_01_28',
          'faster_rcnn_resnet101_coco_2018_01_28',
          'faster_rcnn_inception_resnet_v2_atrous_oid_2018_01_28']

DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.

# List of the strings that is used to add correct label for each box.
COCO_PATH_TO_LABELS = os.path.join('object_detection', 'data', 'mscoco_label_map.pbtxt')
COCO_NUM_CLASSES = 90

OID_PATH_TO_LABELS = os.path.join('object_detection', 'data', 'oid_bbox_trainable_label_map.pbtxt')
OID_NUM_CLASSES = 545

# Path to Test Images:
PATH_TO_TEST_IMAGES_DIR = os.path.join('object_detection', 'test_images')
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image_{}.jpg'.format(i)) for i in range(0, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 15)

## Download Models
(Executed only, if not already downloaded)

In [ ]:
def download_model(model_name):
    """Download model file and extract frozen detection graph."""
    model_file = model_name + '.tar.gz'
    # Download
    if not os.path.isfile(model_file):
        print('Downloading {} ...'.format(model_file))        
        opener = urllib.request.URLopener()
        opener.retrieve(DOWNLOAD_BASE + model_file, model_file)
        # Extract
        tar_file = tarfile.open(model_file)
        for file in tar_file.getmembers():
            file_name = os.path.basename(file.name)
            if 'frozen_inference_graph.pb' in file_name:
                tar_file.extract(file, os.getcwd())        
    else:
        print('Skip downloading {}. Already existing.'.format(model_file))

In [ ]:
for model_name in MODELS:
    download_model(model_name)
print('Done.')

## Load a (frozen) Tensorflow model into memory.

In [ ]:
def load_graph(model_name):
    path_to_ckpt = model_name + '/frozen_inference_graph.pb'
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(path_to_ckpt, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return detection_graph

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
def wrapper(func, *args, **kwargs):
    """Wrapper function for use with timeit.
    See: https://www.pythoncentral.io/time-a-python-function/"""
    def wrapped():
         return func(*args, **kwargs)
    return wrapped

## Detect object in a single image

In [ ]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
              ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                  tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

## Cycle through all test-images, analyse and output with boxes

In [ ]:
def analyze_images(detection_graph):
    for image_path in TEST_IMAGE_PATHS:
        image = Image.open(image_path)
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        output_dict = run_inference_for_single_image(image_np, detection_graph)
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=3)
        plt.figure(figsize=IMAGE_SIZE)
        plt.imshow(image_np)

# Main

In [ ]:
for model_name in MODELS:
    print('Analyzing {} test images using "{}" ...'.format(len(TEST_IMAGE_PATHS), model_name) )
    
    # Label maps map indices to category names, so that when our convolution network predicts 5, we know that this corresponds to airplane.
    if '_coco_' in model_name:
        label_map = label_map_util.load_labelmap(COCO_PATH_TO_LABELS)
        categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=COCO_NUM_CLASSES, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)    
    elif '_oid_' in model_name:
        label_map = label_map_util.load_labelmap(OID_PATH_TO_LABELS)
        categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=OID_NUM_CLASSES, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)    
    else:
        print('Labels couldn\'t be loaded. Abort.')
        continue
    
    # Load model
    detection_graph = load_graph(model_name)
    
    # Analyse & output with timeit, for perfomance measuring
    wrapped = wrapper(analyze_images, detection_graph)
    duration = timeit.timeit(wrapped, number=1)
    print(('Time needed: {}. Average per image: {}').format(duration, duration/len(TEST_IMAGE_PATHS)))